In [1]:
# Step 1: Import all necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib
import os

print("Libraries imported successfully!")

# Step 2: Define Parameters and Fetch Data
STOCK_TICKER = 'AAPL'
TRAIN_START_DATE = '2015-01-01'
TRAINING_DATA_RATIO = 0.8

# Fetch data using yfinance
df = yf.download(STOCK_TICKER, start=TRAIN_START_DATE, progress=False)

# Use only 'Close' price
data = df[['Close']]
dataset = data.values  # Convert DataFrame to numpy array

# Calculate number of rows to train the model
training_data_len = int(np.ceil(len(dataset) * TRAINING_DATA_RATIO))

print(f"Data fetched for {STOCK_TICKER}.")
print(f"Total data points: {len(dataset)}")
print(f"Training data points: {training_data_len}")

# Step 3: Scale the Data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

print("Data scaled successfully.")

# Step 4: Create the Training Dataset
SEQUENCE_LENGTH = 60
train_data = scaled_data[0:int(training_data_len), :]

x_train = []
y_train = []

for i in range(SEQUENCE_LENGTH, len(train_data)):
    x_train.append(train_data[i-SEQUENCE_LENGTH:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape for LSTM [samples, timesteps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

print(f"Training data prepared. Shape of x_train: {x_train.shape}")

# Step 5: Build the LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(x_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Step 6: Train the Model
print("\nStarting model training...")
model.fit(x_train, y_train, batch_size=32, epochs=10)
print("\nModel training complete.")

# Step 7: Save the Trained Model and Scaler
# ✅ FIX: Use a relative path for portability.
# This assumes you run the script from the project's root directory.
# The models will be saved in a 'models' folder in that directory.
MODELS_DIR = 'models'
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

model_path = os.path.join(MODELS_DIR, 'lstm_model.h5')
scaler_path = os.path.join(MODELS_DIR, 'scaler.gz')

model.save(model_path)
joblib.dump(scaler, scaler_path)

print(f"\nModel saved to {model_path}")
print(f"Scaler saved to {scaler_path}")

Libraries imported successfully!


C:\Users\saial\AppData\Local\Temp\ipykernel_25296\2452058338.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(STOCK_TICKER, start=TRAIN_START_DATE, progress=False)


Data fetched for AAPL.
Total data points: 2713
Training data points: 2171
Data scaled successfully.
Training data prepared. Shape of x_train: (2111, 60, 1)


D:\Infosys_springboard\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

...

63/66 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 5.1216e-04

66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 5.1469e-04

66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 5.6409e-04



Model training complete.



Model saved to models\lstm_model.h5
Scaler saved to models\scaler.gz
